In [1]:
import numpy as np
from matplotlib import pyplot as plt
import py_gauge_mc
from tqdm import tqdm

In [2]:
replicas = 128
staging = 8
v_n = 64
L = 4
lf = py_gauge_mc.WindingNumberLeapfrog(replicas + staging, v_n, L,L,L,L)
lf.set_use_heatbath(True)

potentials_form = np.array([x**2 for x in range(0,v_n)], dtype=np.float32)

num_steps = 16
inv_ks = np.linspace(0.1,1.0,num_steps*(replicas - 1) + 1, dtype=np.float32)

In [10]:
# Time trial for seeding updates for i=0
i = 0
sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)
lf.init_potentials(potentials, windings=np.array([[0,0,0,0,0,0],[1,0,0,0,0,0]], dtype=np.int32),num_staging=staging)

def run():
    lf.repeated_seed(1)
    lf.wait_for_gpu()

%timeit run()

612 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def run():
    lf.seed_random_winding(True)
    lf.wait_for_gpu()

%timeit run()

106 µs ± 8.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
def run():
    lf.run_global_sweep()
    lf.wait_for_gpu()

%timeit run()

106 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
def run():
    lf.simulate_local(1)
    lf.wait_for_gpu()

%timeit run()

221 µs ± 3.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
def run():
    lf.simulate_local(10)
    lf.wait_for_gpu()

%timeit run()

1.92 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
def run():
    lf.run_parallel_tempering(False)
    lf.wait_for_gpu()

%timeit run()

3.76 ms ± 65 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
def run():
    lf.run_parallel_tempering(True)
    lf.wait_for_gpu()

%timeit run()

3.82 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
def run():
    lf.get_energies()
    lf.wait_for_gpu()

%timeit run()

1.94 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
def run():
    lf.get_windings()
    lf.wait_for_gpu()

%timeit run()

2.05 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
all_inv_ks = []
all_ws = []
all_es = []

windings = None
counts = None
for i in tqdm(range(num_steps)):
    sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
    potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)
    lf.init_potentials(potentials, windings=windings, standarize=True, num_staging=staging)
    lf.repeated_seed(100)
    w, e = lf.repeated_seed_and_measure(1000)
    windings = w[-1,:,:]
    all_inv_ks.append(sub_inv_ks)
    all_ws.append(w)
    all_es.append(e)
    break

In [ ]:
for ik, e in zip(all_inv_ks, all_es):
    plt.errorbar(ik, e.mean(axis=0), yerr=e.std(axis=0)/np.sqrt(e.shape[0]))
plt.grid()
plt.show()

In [ ]:
for ik, e in zip(all_inv_ks, all_es):
    plt.plot(ik, e.var(axis=0))
plt.grid()
plt.show()

In [ ]:
for ik, w in zip(all_inv_ks, all_ws):
    plt.plot(ik, w.var(axis=(-1,-3)))
plt.grid()
plt.show()